In [ ]:
# EFFR
# https://www.newyorkfed.org/markets/reference-rates/effr

# Methodology
# https://www.atlantafed.org/cenfis/market-probability-tracker
# https://www.cmegroup.com/articles/2023/understanding-the-cme-group-fedwatch-tool-methodology.html

# Settlements FFF
# https://my.apps.factset.com/workstation/navigator/company-security/futures-montage/FF00-USA

# FED meetings
# https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm

In [2]:
from EcoWatch.NelsonSiegelCurvature import NSC
from EcoWatch.Scraping import tbond
import pandas as pd
import numpy as np

In [3]:
tbond_daiy = tbond('2020', '2025')
tbond_daiy = tbond_daiy.drop(['20 Yr', '30 Yr', '1.5 Month'], axis=1)
tbond_daiy = tbond_daiy.interpolate(axis=1)
tbond_daiy = tbond_daiy.dropna(axis=0)

# Define the maturities of the US Treasury Bonds
maturities = np.unique([
    int(col.split()[0]) / 12 if "Mo" in col else int(col.split()[0])
    for col in tbond_daiy.columns
])
# Define the curve maturities
curve_maturities = np.arange(start=maturities.min(), stop=maturities.max()+maturities.min(), step=maturities.min()).round(4)

# Define parameter bounds for optimization: 
bounds = [(0, 1), (-1, 1), (-1, 1), (0, 5)] # Intercept (β0), Slope (β1), Curvature (β2), and Lambda (λ)
x0 = [0.01, 0.01, 0.01, 0.5] # Initial guess for the optimization algorithm
method = 'trust-constr' # Optimization method used for minimization

# Initialisation du modèle
NelsonSiegelCurvature = NSC(maturities=maturities, bounds=bounds, x0=x0, method=method)

# Ajustement des paramètres Nelson-Siegel
nsc_df = NelsonSiegelCurvature.fit(yields=tbond_daiy)

Fitting Nelson-Siegel: 665it [01:01, 12.42it/s]C:\Users\Matteo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\optimize\_differentiable_functions.py:316: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)
Fitting Nelson-Siegel: 791it [01:12, 11.71it/s]C:\Users\Matteo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\optimize\_differentiable_functions.py:316: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self

In [4]:
nsc_df.reset_index().to_csv('nsc.csv', index=False)

In [24]:
buckets = {
    "0.00–0.25": 0.125,
    "0.25–0.50": 0.375,
    "0.50–0.75": 0.625,
    "0.75–1.00": 0.875,
    "1.00–1.25": 1.125,
    "1.25–1.50": 1.375,
    "1.50–1.75": 1.625,
    "1.75–2.00": 1.875,
    "2.00–2.25": 2.125,
    "2.25–2.50": 2.375,
    "2.50–2.75": 2.625,
    "2.75–3.00": 2.875,
    "3.00–3.25": 3.125,
    "3.25–3.50": 3.375,
    "3.50–3.75": 3.625,
    "3.75–4.00": 3.875,
    "4.00–4.25": 4.125,
    "4.25–4.50": 4.375,
    "4.50–4.75": 4.625,
    "4.75–5.00": 4.875,
    "5.00–5.25": 5.125,
    "5.25–5.50": 5.375,
    "5.50–5.75": 5.625,
    "5.75–6.00": 5.875,
    "6.00–6.25": 6.125,
    "6.25–6.50": 6.375,
    "6.50–6.75": 6.625,
    "6.75–7.00": 6.875,
}

In [47]:
effr = pd.read_csv('overnight_rates.csv', index_col='Effective Date')
effr.index = pd.to_datetime(effr.index, format='mixed')
effr = effr[effr['Rate Type'] == 'EFFR']['Rate (%)']
effr = effr.sort_index()

In [71]:
import numpy as np
from scipy.optimize import minimize

# Buckets de taux (en %)
rate_buckets = np.array([4.75, 4.00, 4.25, 4.50])
n_buckets = len(rate_buckets)

# Taux implicites observés pour 3 mois (en %) via les futures FFJ25, FFK25, FFM25
observed_rates = np.array([4.323, 4.275])
n_contracts = len(observed_rates)

# Construction de la matrice des contraintes
# Chaque ligne = taux moyen du mois = somme pondérée des buckets
A = np.tile(rate_buckets, (n_contracts, 1))
b = observed_rates

# Ajout de la contrainte de somme des probabilités = 1
A_eq = np.vstack([A, np.ones(n_buckets)])
b_eq = np.append(b, 1)

# Fonction objectif : minimiser ||p||² (distribution la plus "lisse")
def objective(p):
    return np.sum(p**2)

# Contrainte d’égalité : A_eq @ p = b_eq
constraints = {
    'type': 'eq',
    'fun': lambda p: A_eq @ p - b_eq
}

# Contrainte de positivité : p_i ≥ 0
bounds = [(0, 1)] * n_buckets

# Initialisation : distribution uniforme
p0 = np.ones(n_buckets) / n_buckets

# Définir les options pour l'optimisation
options = {
    'maxiter': 100000,
    'disp': True
}

# Optimisation avec méthode 'trust-constr'
result = minimize(objective, p0, method='trust-constr', bounds=bounds, constraints=constraints, options=options)

# Résultat final
if result.success:
    probas = result.x
    for r, p in zip(rate_buckets, probas):
        print(f"Taux {r:.2f} % : probabilité = {p:.4f}")
else:
    print("Optimisation échouée :", result.message)


`xtol` termination condition is satisfied.
Number of iterations: 48107, function evaluations: 241110, CG iterations: 48094, optimality: 9.87e-11, constraint violation: 2.40e-02, execution time: 4e+01 s.
Taux 4.75 % : probabilité = 0.1588
Taux 4.00 % : probabilité = 0.3412
Taux 4.25 % : probabilité = 0.2804
Taux 4.50 % : probabilité = 0.2196
